### Naive Bayes

En este notebook entrenaremos tres modelos de MultinomialNB, uno con el mismo preprocesamiento del dataset realizado en la primera parte del TP, el segundo con la función `preprocessing_significantes`, y el último con el `preprocessing_equilibrado`, para estudiar cómo se comporta este modelo ante esas tres distribuciones.

In [2]:
# Modelo
from sklearn.naive_bayes import GaussianNB, CategoricalNB, MultinomialNB
from sklearn.naive_bayes import BernoulliNB, ComplementNB

# Preprocessing
import utils
import preprocessing
from sklearn.preprocessing import MinMaxScaler

# Búsqueda
from sklearn.model_selection import train_test_split, GridSearchCV

# Metrics
from sklearn.metrics import plot_roc_curve, roc_auc_score
from sklearn.metrics import classification_report, plot_confusion_matrix

# Plots
from matplotlib import pyplot as plt
import seaborn as sns

# Otros
import pandas as pd
import numpy as np

Como siempre, comencemos por obtener el dataset

In [3]:
df = utils.get_data()

### Preprocessing base

En este modelo en particular, nos interesa entrenar modelos con distintos valores del hiperparámetro alpha, que haremos variar entre 0 y 1.

In [4]:
params = { 'alpha': np.linspace(0,1,1000) }
model = MultinomialNB()

In [5]:
gscv = GridSearchCV(model, params, scoring='roc_auc', n_jobs=-1, verbose=10)

Separamos los datos, los preprocesamos con la primer función que mostraremos en este notebook, y escalamos los valores obtenidos entre 0 y 1 antes de entrenar.

In [6]:
X_train_1, X_test_1, y_train_1, y_test_1 = train_test_split(df.drop('tiene_alto_valor_adquisitivo',1), df['tiene_alto_valor_adquisitivo'], random_state=112, stratify=df['tiene_alto_valor_adquisitivo'])
X_train_1, X_test_1 = preprocessing.preprocessing_base_parte_1(X_train_1, X_test_1)

In [7]:
scaler = MinMaxScaler() # Por default, en [0,1]

X_train_1 = pd.DataFrame(scaler.fit_transform(X_train_1))
X_test_1 = pd.DataFrame(scaler.transform(X_test_1))

In [ ]:
%%time
gscv.fit(X_train_1, y_train_1)
score = roc_auc_score(y_test_1, gscv.predict_proba(X_test_1)[:,1])

Fitting 5 folds for each of 1000 candidates, totalling 5000 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:    2.6s
[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:    2.9s
[Parallel(n_jobs=-1)]: Done  16 tasks      | elapsed:    3.1s
[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed:    3.3s
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:    3.4s
[Parallel(n_jobs=-1)]: Done  45 tasks      | elapsed:    3.6s
[Parallel(n_jobs=-1)]: Done  56 tasks      | elapsed:    3.8s
[Parallel(n_jobs=-1)]: Done  69 tasks      | elapsed:    3.9s
[Parallel(n_jobs=-1)]: Done  82 tasks      | elapsed:    4.2s
[Parallel(n_jobs=-1)]: Done  97 tasks      | elapsed:    4.6s
[Parallel(n_jobs=-1)]: Done 112 tasks      | elapsed:    4.8s
[Parallel(n_jobs=-1)]: Done 129 tasks      | elapsed:    5.1s
[Parallel(n_jobs=-1)]: Batch computation too fast (0.1979s.) Setting batch_size=2.
[Parallel(n_jobs=-1)]: Done 146 tasks      | elapsed:    5.4s
[Parallel(n_jobs=-1)]: Done 170 tas

In [ ]:
gscv.best_params_

In [ ]:
bayes_base = gscv.best_estimator_
bayes_base.fit(X_train_1, y_train_1)

In [ ]:
score

In [ ]:
print(classification_report(y_test_1,bayes_base.predict(X_test_1)))

Vemos que, si bien la métrica AUC-ROC arrojó un buen resultado, el accuracy es bastante bajo, y las demás métricas nos dan una idea de que el modelo seleccionado no generaliza nada bien.

In [ ]:
fig, ax = plt.subplots(figsize=(15,7))
plt.grid(False)
plot_confusion_matrix(bayes_base, X_test_1, y_test_1, cmap=plt.cm.Blues, display_labels=['0', '1'], ax=ax)
plt.show()

Si bien, a priori, parece dar buenos resultados para los True Positive y False Negative, vemos que los True Negative son muy escasos y tiene una cantidad indeseablemente alta de False Positive.

In [ ]:
plot_roc_curve(bayes_base, X_test_1, y_test_1)

La forma de la curva y su lento crecimiento en el eje vertical nos ayuda a reforzar la idea de que no obtuvimos un buen clasificador.

### Preprocessing Significantes

Repetimos el procedimiento anterior, esta vez trabajando los datos con la función de preprocessing significantes para con un 90% de varianza explicada.

In [ ]:
df = utils.get_data()

In [ ]:
params = { 'alpha': np.linspace(0,1,1000) }
model = MultinomialNB()

In [ ]:
gscv = GridSearchCV(model, params, scoring='roc_auc', n_jobs=-1, verbose=10)

In [ ]:
X = df[df.columns.drop('tiene_alto_valor_adquisitivo')]
Y = df['tiene_alto_valor_adquisitivo']
X = pd.get_dummies(X, drop_first=True)

X_train_2, X_test_2, y_train_2, y_test_2 = train_test_split(X, Y, random_state=112, stratify=Y)
X_train_2, X_test_2 = preprocessing.preprocessing_significantes(X_train_2, X_test_2, 0.9)

In [ ]:
scaler = MinMaxScaler() # Por default, en [0,1]

X_train_2 = pd.DataFrame(scaler.fit_transform(X_train_2))
X_test_2 = pd.DataFrame(scaler.transform(X_test_2))

In [ ]:
%%time
gscv.fit(X_train_2, y_train_2)
score = roc_auc_score(y_test_2, gscv.predict_proba(X_test_2)[:,1])

In [ ]:
gscv.best_params_

Antes de continuar, notemos que el alpha obtenido en este caso es muy chico, en comparación con el seleccionado para el modelo anterior que era bastante cercano a 1.

In [ ]:
score

In [ ]:
bayes_significantes = gscv.best_estimator_
bayes_significantes.fit(X_train_2, y_train_2)

Vemos una mejora en la métrica AUC-ROC

In [ ]:
print(classification_report(y_test_2,bayes_significantes.predict(X_test_2)))

Nuevamente, a pesar de haber obtenido un resultado decente según AUC-ROC, vemos que la clasificación es muy mala.

In [ ]:
fig, ax = plt.subplots(figsize=(15,7))
plt.grid(False)
plot_confusion_matrix(bayes_significantes, X_test_2, y_test_2, cmap=plt.cm.Blues, display_labels=['0', '1'], ax=ax)
plt.show()

Vemos que el comportamiento es incluso peor que el modelo anterior.

In [ ]:
plot_roc_curve(bayes_significantes, X_test_2, y_test_2)

### Preprocessing Equilibrado

Finalizaremos este análisis intentando obtener un mejor resultado al equilibrar la cantidad de muestras con bajo poder adquisitivo a la de muestras con alto poder adquisitivo.

In [ ]:
df = utils.get_data()

In [ ]:
params = { 'alpha': np.linspace(0,1,1000) }
model = MultinomialNB()

In [ ]:
gscv = GridSearchCV(model, params, scoring='roc_auc', n_jobs=-1, verbose=10)

In [ ]:
X = df[df.columns.drop('tiene_alto_valor_adquisitivo')]
Y = df['tiene_alto_valor_adquisitivo']
X = pd.get_dummies(X)

X_train_3, X_test_3, y_train_3, y_test_3 = train_test_split(X, Y, random_state=112, stratify=Y)
X_train_3, X_test_3, y_train_3, y_test_3 = preprocessing.preprocessing_equilibrado(X_train_3, X_test_3, y_train_3, y_test_3)

In [ ]:
scaler_train = MinMaxScaler() # Por default, en [0,1]
scaler_test = MinMaxScaler() # Por default, en [0,1]

X_train_3 = pd.DataFrame(scaler_train.fit_transform(X_train_3))
X_test_3 = pd.DataFrame(scaler_test.fit_transform(X_test_3))

In [ ]:
%%time
gscv.fit(X_train_3, y_train_3)

In [ ]:
gscv.best_params_

Vemos que el alpha seleccionado está casi en el límite superior del rango elegido.

In [ ]:
bayes_equilibrado = gscv.best_estimator_
bayes_equilibrado.fit(X_train_3, y_train_3)

In [ ]:
print(classification_report(y_test_3,bayes_equilibrado.predict(X_test_3)))

Si bien sigue sin ser un clasificador excelente, vemos una mejora sustancial respecto de los anteriores, al menos en términos de la generalización. Sin embargo, el accuracy empeoró.

In [ ]:
fig, ax = plt.subplots(figsize=(15,7))
plt.grid(False)
plot_confusion_matrix(bayes_equilibrado, X_test_3, y_test_3, cmap=plt.cm.Blues, display_labels=['0', '1'], ax=ax)
plt.show()

A pesar de haber mejorado en algunos aspectos, ahora tenemos un número demasiado alto en los False Negatives

In [ ]:
plot_roc_curve(bayes_equilibrado, X_test_3, y_test_3)

Nuevamente, vemos que los falsos positivos crecen muy rápido en comparación con los verdaderos positivos, haciendo que la clasificación sea tan pobre como vimos.

A modo de conclusión general, no vemos que MultinomialNB sea un modelo interesante para los objetivos de este trabajo.

# Persistiendo los modelos

In [ ]:
import pickle

In [ ]:
pickle.dump(bayes_base, open("Modelos/MultinomialNB/bayes_base.pickle", "wb"))
pickle.dump(bayes_significantes, open("Modelos/MultinomialNB/bayes_significantes.pickle", "wb"))
pickle.dump(bayes_equilibrado, open("Modelos/MultinomialNB/bayes_equilibrado.pickle", "wb"))

# Predicción de HoldOut

Usaremos para predecir el NaiveBayes obtenido para el preprocessing significantes de 90%

In [ ]:
bayes_significantes = pickle.load(open("Modelos/MultinomialNB/bayes_significantes.pickle", "rb"))

In [ ]:
def holdout():
    dfv = utils.get_data()
    Xv = dfv[dfv.columns.drop('tiene_alto_valor_adquisitivo')]
    Xv = pd.get_dummies(Xv, drop_first=True)
    Xv, _, _, _ = train_test_split(Xv, dfv['tiene_alto_valor_adquisitivo'], random_state=112)

    ids, X = utils.get_holdout_data()
    X = pd.get_dummies(X, drop_first=True)

    notInHoldout=[]
    for c in Xv.columns:
        if c not in X.columns:
            notInHoldout.append(c)
    X[notInHoldout] = 0

    Xv, X = preprocessing.preprocessing_significantes(Xv, X, variance=0.9)
    return ids, X

In [ ]:
ids, X = holdout()

In [ ]:
utils.escribir_holdout(bayes_significantes.predict(X), "3 - MultinomialNB", ids)